In [3]:
#importing Video
import os
from os import path
import cv2 
import math
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing import image
import numpy as np
from keras.utils import np_utils
from skimage.transform import resize 
import glob
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from IPython.core.debugger import Tracer; debug_here = Tracer()
#creating a dataframe of video names
data_csv = pd.read_csv("C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\dfdc_train_part_46.csv") 
names_csv = data_csv.iloc[:,0]
video_names = names_csv.str.split('.').str[0]
data_csv.set_index("_key",inplace=True)
for i in names_csv:
#    label = data_csv.loc[[i],['label']]
    label = label.append((data_csv.loc[[i],['label']]))
label.iloc[42:44]

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "c:\users\vardh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\vardh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\users\vardh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "c:\users\vardh\appdata\local\programs\python\python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "c:\users\vardh\appdata\local\programs\python\python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load fai

In [ ]:
#Extracting Frames from video into a file
video_name = video_names[42:44]
print(video_name)
file_exists = path.exists('C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\train_1')
print(file_exists)
if file_exists == False:
    os.mkdir('C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\train_1')
for i in video_name:
    cap = cv2.VideoCapture('C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\dfdc_train_part_46\\'+i+'.mp4')
    count = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            name = 'train_1+'+i+'+%d.jpg'% count;count+=1
#            print('creating....' + name)
            ret1 = cv2.imwrite(os.path.join('C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\train_1',name),frame)
        else:
            break    
cap.release()
cv2.destroyAllWindows()

In [ ]:
#merging images and their labels(Real/Fake) into a csv file
images = glob('C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\train_1\\train_1+*.jpg')
train_image = []
train_class = []
label.head()
#train_image.append(images[1].split('+')[1])
count = 0
for i in tqdm(range(len(images))):      
    n = images[i].split('+')[1]
#    n3 = n
    if n != n3:
        count = 0
    n3 = n
    n1 = n+'%d.jpg'% count;count+=1
#    train_image.append(images[i].split('+')[1])
    train_image.append(n1)
#    k = images[i].split('+')[1]+'.mp4'
    k = n+'.mp4'
#    print(k)
    g = label.loc[[k]]
    g1 = g.iloc[0]['label']
    train_class.append(g1)
#print(train_image)
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class
train_data.to_csv('C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\train_data.csv',header=True,index=False)

In [ ]:
#feature extraction
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
train = pd.read_csv('C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\train_data.csv')
#print(train.head())
#store frames in list
train_image = []
for i in train.iloc[:,0]:
    num = 0
    name ='train_1+'+i 
    name = name[:-5]+'+%d.jpg'% num;num+=1
    img = glob('C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\train_1\\train_1+*.jpg')
for i in img:
    num+=1
    imag = image.load_img(i, target_size=(224,224,3))
    imag = np.array(imag)
    imag = imag/255
    train_image.append(imag)
#    if num == 5:
#        break
X = np.array(train_image)
X.shape

In [ ]:
#Creating a validation set
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
Y = train['class']
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.66, random_state = 7)
base_model = VGG16(weights='imagenet',include_top = False)
base_model.compile(optimizer=keras.optimizers.RMSprop(),
                   loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True))
X_train = base_model.predict(X_train)
X_test = base_model.predict(X_test)

In [13]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
Y_train = pd.get_dummies(Y_train)
Y_test = pd.get_dummies(Y_test)
Y_train.shape
#from keras.utils import to_categorical
#Y_train = to_categorical(Y_train)
#Y_test = to_categorical(Y_test)
#sklearn.preprocessing.OneHotEncoder(categories='3')

(200, 2)

In [15]:
# Reshaping the data into single dimension and normalizing
print(X_train.shape)
print(X_test.shape)
X_train = X_train.reshape(200,7*7*512)
X_test = X_test.reshape(391,7*7*512)
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

(200, 25088)
(391, 25088)


In [16]:
#Creating an architecture using Sequential
from keras.layers import Activation
model = Sequential()
model.add(Dense(1024,activation='relu',input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

In [17]:
#Training the model and saving the weights and compiling the model before using it
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5',save_best_only=True,monitor= 'val_loss',mode='min')
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
final_model = model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test), callbacks=[mcp_save], batch_size=128)
final_model = model.fit(X_train,Y_train)

Train on 200 samples, validate on 391 samples
Epoch 1/10
200/200 [==============================] - 2s 10ms/step - loss: 0.7234 - accuracy: 0.4800 - val_loss: 0.6279 - val_accuracy: 0.7545
Epoch 2/10
200/200 [==============================] - 1s 6ms/step - loss: 0.7906 - accuracy: 0.5150 - val_loss: 0.5973 - val_accuracy: 0.4987
Epoch 3/10
200/200 [==============================] - 1s 6ms/step - loss: 0.7560 - accuracy: 0.5600 - val_loss: 0.5057 - val_accuracy: 1.0000
Epoch 4/10
200/200 [==============================] - 1s 6ms/step - loss: 0.7169 - accuracy: 0.5600 - val_loss: 0.3935 - val_accuracy: 1.0000
Epoch 5/10
200/200 [==============================] - 1s 6ms/step - loss: 0.5781 - accuracy: 0.7200 - val_loss: 0.2898 - val_accuracy: 1.0000
Epoch 6/10
200/200 [==============================] - 1s 6ms/step - loss: 0.4454 - accuracy: 0.7500 - val_loss: 0.1931 - val_accuracy: 1.0000
Epoch 7/10
200/200 [==============================] - 1s 6ms/step - loss: 0.3622 - accuracy: 0.8250 -

In [2]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.externals.joblib import dump
from sklearn.externals.joblib import load
import pickle
print(X_test.shape)

#pred = model.predict(X_test)
#file_name = 'C:\\Users\\vardh\\Vardhan\\ED\\Videos\\dfdc_train_part_46\\Final_video_model.sav'
#pickle.dump(final_model,open(file_name, 'wb'))
#scores = model.evaluate(X_test,Y_test)
#print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

c:\users\vardh\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


NameError: name 'X_test' is not defined